In [1]:
import argparse
import httplib2
import requests
import pandas as pd
 
from collections import defaultdict
from dateutil import relativedelta
from googleapiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [2]:
# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "165277451909-549djem84js33jh3thql01iccovj6hsl.apps.googleusercontent.com"
CLIENT_SECRET = "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"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

parser = argparse.ArgumentParser(
    formatter_class=argparse.RawDescriptionHelpFormatter,
    parents=[tools.argparser])
flags = parser.parse_args([])
 
flow = client.OAuth2WebServerFlow(client_id=CLIENT_ID,
                           client_secret=CLIENT_SECRET,
                           scope=OAUTH_SCOPE)
 
storage = file.Storage('searchconsole.dat')
 
credentials = storage.get()
if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage, flags)
 
# Create an httplib2.Http object and authorize it with our credentials
http = credentials.authorize(http=httplib2.Http())

In [3]:
webmasters_service = build('searchconsole', 'v1', http)
webmasters_service

In [18]:
website = "https://www.samsung.com/us/"
STARTDATE = '2024-03-01'
ENDDATE = '2024-03-15'

startRow = 0
all_response = []

while (startRow == 0) or (startRow % 25000 == 0):
    request_body = {
        "startDate" : "2024-03-01",
        "endDate" : "2024-03-15",
        "dimensions" : ['Date', 'Page'],
        "rowLimit" : 25000,
        "dateState" : "final",
        "startRow" : startRow
    }
    response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()
    startRow += len(response_data['rows'])
    print("fetched up to " + str(startRow) + " rows of data")
    all_response.extend(response_data['rows'])

data_for_df = []

for row in all_response:
    temp = []
    temp.append(row['keys'][0])  #date
    temp.append(row['keys'][1])  #page
    temp.append(row['clicks'])
    temp.append(row['impressions'])
    temp.append(row['ctr'])
    temp.append(row['position'])
    temp.append(website)
    data_for_df.append(temp)

df = pd.DataFrame(data_for_df, columns = ['date', 'page', 'clicks', 'impressions', 'ctr', 'position', 'property'])
df.to_csv(f'{website.split("/")[-2]}-{STARTDATE}-to-{df['date'].max()}-date-page.csv', index=False)


fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 57751 rows of data


In [6]:
df['date'].max()

'2024-03-14'

In [13]:
#일자별 행수 확인 
df.groupby('date').count()

,page,clicks,impressions,ctr,position,property
date,,,,,,
2024-03-01,4689,4689,4689,4689,4689,4689
2024-03-02,4662,4662,4662,4662,4662,4662
2024-03-03,4688,4688,4688,4688,4688,4688
2024-03-04,4702,4702,4702,4702,4702,4702
2024-03-05,4691,4691,4691,4691,4691,4691
2024-03-06,4676,4676,4676,4676,4676,4676
2024-03-07,4650,4650,4650,4650,4650,4650
2024-03-08,4596,4596,4596,4596,4596,4596
2024-03-09,4560,4560,4560,4560,4560,4560


In [15]:
df.groupby('date').size()

date
2024-03-01    4689
2024-03-02    4662
2024-03-03    4688
2024-03-04    4702
2024-03-05    4691
2024-03-06    4676
2024-03-07    4650
2024-03-08    4596
2024-03-09    4560
2024-03-10    4588
2024-03-11    4630
2024-03-12    4631
2024-03-13    4620
2024-03-14    4629
dtype: int64

In [9]:
website = "https://www.samsung.com/uk/"
startRow = 0

request_body = {
    "startDate" : "2024-03-01",
    "endDate" : "2024-03-01",
    "dimensions" : ['Date', 'Page'],
    "rowLimit" : 25000,
    "dateState" : "final",
    "startRow" : startRow
}
response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

In [10]:
response_data

{'rows': [{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'],
   'clicks': 16774,
   'impressions': 24205,
   'ctr': 0.6929973146044206,
   'position': 4.245197273290643},
  {'keys': ['2024-03-01', 'https://www.samsung.com/uk/'],
   'clicks': 5183,
   'impressions': 64663,
   'ctr': 0.08015402935217976,
   'position': 29.72690719576883},
  {'keys': ['2024-03-01',
    'https://www.samsung.com/uk/try-galaxy-on-your-phone/'],
   'clicks': 5072,
   'impressions': 15647,
   'ctr': 0.32415159455486675,
   'position': 8.850961845721224},
  {'keys': ['2024-03-01',
    'https://www.samsung.com/uk/mobile-phone-buying-guide/introducing-samsung-galaxy-s24/'],
   'clicks': 1246,
   'impressions': 28116,
   'ctr': 0.044316403471333046,
   'position': 8.940140845070424},
  {'keys': ['2024-03-01',
    'https://www.samsung.com/uk/support/mobile-devices/how-do-i-turn-on-the-developer-options-menu-on-my-samsung-galaxy-device/'],
   'clicks': 1039,
   'impressions': 20101,
   'ctr': 0.0516889706

In [11]:
response_data['rows']

[{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'],
  'clicks': 16774,
  'impressions': 24205,
  'ctr': 0.6929973146044206,
  'position': 4.245197273290643},
 {'keys': ['2024-03-01', 'https://www.samsung.com/uk/'],
  'clicks': 5183,
  'impressions': 64663,
  'ctr': 0.08015402935217976,
  'position': 29.72690719576883},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/try-galaxy-on-your-phone/'],
  'clicks': 5072,
  'impressions': 15647,
  'ctr': 0.32415159455486675,
  'position': 8.850961845721224},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/mobile-phone-buying-guide/introducing-samsung-galaxy-s24/'],
  'clicks': 1246,
  'impressions': 28116,
  'ctr': 0.044316403471333046,
  'position': 8.940140845070424},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/support/mobile-devices/how-do-i-turn-on-the-developer-options-menu-on-my-samsung-galaxy-device/'],
  'clicks': 1039,
  'impressions': 20101,
  'ctr': 0.05168897069797523,
  'position': 8.6044972886

In [12]:
len(response_data)

2

In [13]:
len(response_data['rows'])

4689

In [15]:
for row in response_data['rows']:
    print(row)

{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'], 'clicks': 16774, 'impressions': 24205, 'ctr': 0.6929973146044206, 'position': 4.245197273290643}
{'keys': ['2024-03-01', 'https://www.samsung.com/uk/'], 'clicks': 5183, 'impressions': 64663, 'ctr': 0.08015402935217976, 'position': 29.72690719576883}
{'keys': ['2024-03-01', 'https://www.samsung.com/uk/try-galaxy-on-your-phone/'], 'clicks': 5072, 'impressions': 15647, 'ctr': 0.32415159455486675, 'position': 8.850961845721224}
{'keys': ['2024-03-01', 'https://www.samsung.com/uk/mobile-phone-buying-guide/introducing-samsung-galaxy-s24/'], 'clicks': 1246, 'impressions': 28116, 'ctr': 0.044316403471333046, 'position': 8.940140845070424}
{'keys': ['2024-03-01', 'https://www.samsung.com/uk/support/mobile-devices/how-do-i-turn-on-the-developer-options-menu-on-my-samsung-galaxy-device/'], 'clicks': 1039, 'impressions': 20101, 'ctr': 0.05168897069797523, 'position': 8.604497288692105}
{'keys': ['2024-03-01', 'https://www.samsung.com/uk

In [16]:
all_response = []

for row in response_data['rows']:
    all_response.append(row)

all_response

[{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'],
  'clicks': 16774,
  'impressions': 24205,
  'ctr': 0.6929973146044206,
  'position': 4.245197273290643},
 {'keys': ['2024-03-01', 'https://www.samsung.com/uk/'],
  'clicks': 5183,
  'impressions': 64663,
  'ctr': 0.08015402935217976,
  'position': 29.72690719576883},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/try-galaxy-on-your-phone/'],
  'clicks': 5072,
  'impressions': 15647,
  'ctr': 0.32415159455486675,
  'position': 8.850961845721224},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/mobile-phone-buying-guide/introducing-samsung-galaxy-s24/'],
  'clicks': 1246,
  'impressions': 28116,
  'ctr': 0.044316403471333046,
  'position': 8.940140845070424},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/support/mobile-devices/how-do-i-turn-on-the-developer-options-menu-on-my-samsung-galaxy-device/'],
  'clicks': 1039,
  'impressions': 20101,
  'ctr': 0.05168897069797523,
  'position': 8.6044972886

In [17]:
response_data['rows']

[{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'],
  'clicks': 16774,
  'impressions': 24205,
  'ctr': 0.6929973146044206,
  'position': 4.245197273290643},
 {'keys': ['2024-03-01', 'https://www.samsung.com/uk/'],
  'clicks': 5183,
  'impressions': 64663,
  'ctr': 0.08015402935217976,
  'position': 29.72690719576883},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/try-galaxy-on-your-phone/'],
  'clicks': 5072,
  'impressions': 15647,
  'ctr': 0.32415159455486675,
  'position': 8.850961845721224},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/mobile-phone-buying-guide/introducing-samsung-galaxy-s24/'],
  'clicks': 1246,
  'impressions': 28116,
  'ctr': 0.044316403471333046,
  'position': 8.940140845070424},
 {'keys': ['2024-03-01',
   'https://www.samsung.com/uk/support/mobile-devices/how-do-i-turn-on-the-developer-options-menu-on-my-samsung-galaxy-device/'],
  'clicks': 1039,
  'impressions': 20101,
  'ctr': 0.05168897069797523,
  'position': 8.6044972886

In [18]:
type(all_response)

list

In [19]:
type(response_data['rows'])

list

In [20]:
all_response[0]

{'keys': ['2024-03-01', 'https://www.samsung.com/uk/boost/'],
 'clicks': 16774,
 'impressions': 24205,
 'ctr': 0.6929973146044206,
 'position': 4.245197273290643}

In [21]:
all_response[0]['keys']

['2024-03-01', 'https://www.samsung.com/uk/boost/']

In [22]:
all_response[0]['keys'][0]

'2024-03-01'

In [23]:
all_response[0]['clicks']

16774

각 일자별 행수가 몇개인지 확인하는 방법?